In [ ]:
#primary requirements: # python3, numpy, pandas, xarray, dask, bottleneck, netCDF4, cftime, matplotlib, marineHeatWaves
# run this script to check if all dependencies are installed and working
# also fixes np.NaN to np.nan in marineHeatWaves module

import sys, platform
import numpy as np
import pandas as pd
import xarray as xr, dask, bottleneck, netCDF4, cftime
from pathlib import Path
from typing import List, Optional, Tuple, Dict, Union
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import marineHeatWaves as mhw, re, pathlib
from importlib.metadata import version, PackageNotFoundError

# convert np.NAN to np.nan to avoid dependency issues
if not hasattr(np, "NaN"):
    np.NaN = np.nan  

#Check the package is working
try:
    mhw_ver = version('marineHeatWaves')
except PackageNotFoundError:
    mhw_ver = getattr(mhw, '__version__', 'unknown')
print('OK:', pd.__version__, xr.__version__, dask.__version__, 'mhw:', mhw_ver)
print('mhw module file:', getattr(mhw, '__file__', 'unknown'))

#checkk the marineHeatWaves module file and replace np.NaN with np.nan
p = pathlib.Path(mhw.__file__)
p.write_text(re.sub(r'\bnp\.NaN\b', 'np.nan', p.read_text()))

print(sys.executable)
print(platform.python_version())


In [ ]:
#check if marineHeatWaves package is working

t = pd.date_range("2000-01-01", periods=40, freq="D")
ords = np.array([d.toordinal() for d in t], dtype=int)
temp = 25 + np.sin(np.linspace(0, 6.28, 40)); temp[15:25] += 2

res, clim = mhw.detect(ords, temp, climatologyPeriod=[2000, 2000], pctile=90, minDuration=5, joinAcrossGaps=True) #hobdey's defination
print("events:", len(res.get("time_start", [])))

# Verifies that core libs are importable and compatible with marineHeatWaves.
from importlib import import_module
import numpy as np
import marineHeatWaves as mhw

want = ["numpy", "pandas", "xarray", "dask", "bottleneck","netCDF4", "cftime", "marineHeatWaves"]
vers = {}
problems = []

for m in want:
    try:
        import_module(m)
        try:
            vers[m] = version(m)
        except PackageNotFoundError:
            mod = import_module(m)
            vers[m] = getattr(mod, "__version__", "unknown")
    except Exception as e:
        problems.append((m, str(e)))
        vers[m] = "IMPORT FAILED"

print("Package versions detailes:")
for k in want:
    print(f"{k:16s} : {vers[k]}")
if problems:
    print("\n[ENV WARNING] Some packages failed to import:")
    for m, msg in problems:
        print(f" - {m}: {msg}")

# marineHeatWaves.detect accepts ordinal ints + 1D temp array
try:
    # make a synthetic 40-day series with a warm spell
    days = np.array([pd.Timestamp("2000-01-01") + pd.Timedelta(i, "D") for i in range(40)])
    ords = np.array([d.toordinal() for d in days], dtype=int)
    temp = 25 + np.sin(np.linspace(0, 6.28, 40))
    temp[15:25] += 2.0  # warm event
    res, clim = mhw.detect(ords, temp, climatologyPeriod=[1991, 2020], pctile=90, minDuration=5, joinAcrossGaps=True)
    print("\nmarineHeatWaves.detect is working correctly.")
    print(f"Detected events: {len(res.get('time_start', []))}")
except Exception as e:
    print("\n[ENV ERROR] marineHeatWaves.detect detection failed:", e)


In [ ]:
# Checking the files and Summarize the data

from glob import glob
import numpy as np, pandas as pd

# file path: data loading and visualization
FILES_GLOB = "/home/Desktop/Noah_data_1982-2024_SST_daily_mean/sst.day.mean.*.nc"

# region of interest
ROI = dict(lat_min=0.0, lat_max=30.0, lon_min=40.0, lon_max=110.0)

# North indian ocean subregions
REGIONS = {
    "Arabian Sea":    {"lon_min": 40.0, "lon_max": 78.0,  "lat_min": 0.0, "lat_max": 30.0},
    "Bay Of Bengal":   {"lon_min": 78.0, "lon_max": 110.0, "lat_min": 0.0, "lat_max": 30.0},
    "North Indian Ocean": {"lon_min": 40.0, "lon_max": 110.0, "lat_min": 0.0, "lat_max": 30.0},
}

# IO / plotting
use_dask = True                 # set False to fully load into memory
chunks = {"time": 90} if use_dask else None
sample_map_date: Optional[str] = None  
boxmean_var_name = "sst"       
climatologyPeriod = [1982, 2024]  

# loading the data
def open_sst(files, chunks=None, engine: str = "netcdf4") -> xr.Dataset:
    if isinstance(files, str):
        p = Path(files)
        if any(ch in files for ch in "*?[]"):
            paths = sorted(glob(files))
            if not paths:
                raise FileNotFoundError(f"No files match glob: {files}")
            return xr.open_mfdataset(paths, combine="by_coords", parallel=True, chunks=chunks, engine=engine)
        else:
            if not p.exists():
                raise FileNotFoundError(f"File not found: {files}")
            return xr.open_dataset(files, chunks=chunks, engine=engine)
    else:
        paths = [str(Path(f)) for f in files]
        for f in paths:
            if not Path(f).exists():
                raise FileNotFoundError(f"File not found: {f}")
        return xr.open_mfdataset(paths, combine="by_coords", parallel=True, chunks=chunks, engine=engine)

# Subset to region of interest
def subset_roi(ds: xr.Dataset, roi: Dict[str, float], var: str = "sst") -> xr.Dataset:
    # Make sure coords are named commonly
    lat_name = "lat" if "lat" in ds.coords else "latitude"
    lon_name = "lon" if "lon" in ds.coords else "longitude"

    ds2 = ds.sel(**{lat_name: slice(roi["lat_min"], roi["lat_max"]),lon_name: slice(roi["lon_min"], roi["lon_max"]),})
    # Keep only the variable of interest and coords
    if var in ds2:
        return ds2[[var]]
    else:
        raise KeyError(f"Variable '{var}' not found. Available: {list(ds2.data_vars)}")

# Print metadata
def print_metadata(ds: xr.Dataset, var: str = "sst") -> None:
    # Dimention sizes
    for c in ds.coords:
        arr = ds.coords[c]
        try:
            vals = arr.values
            preview = f"{vals[:3]} ... {vals[-3:]}" if vals.size > 6 else vals
        except Exception:
            preview = arr
        print(f"{c}: {preview}")

    # convert time 
    if "time" in ds.coords:
        t = pd.to_datetime(ds["time"].values)
        print("\nTime Coverage:")
        print(f"Start: {pd.Timestamp(t[0]).date()}, End: {pd.Timestamp(t[-1]).date()}, Length: {t.size} steps")

    # Global attrs
    print("\nGlobal attributes:")
    for k, v in ds.attrs.items():
        print(f"{k}: {v}")

    # Variable attrs
    if var in ds:
        print(f"\nVariable '{var}' attributes:")
        for k, v in ds[var].attrs.items():
            print(f"{k}: {v}") 
            print("Dimensions:")
    for k, v in ds.dims.items():
        print(f"{k}: {v}")

    # Coordinates preview
    print("\nCoordinates (first few):")
   
def summarize_events_table(res: dict) -> pd.DataFrame:
    """
    Convert marineHeatWaves 'res' dict to a tidy event table.'time_*' in res are *ordinal* days (Python datetime.toordinal).
    """
    to_ts = lambda arr: pd.to_datetime([pd.Timestamp.fromordinal(int(d)) for d in np.asarray(arr)])

    return pd.DataFrame({
        "start_date":                   to_ts(res["time_start"]),
        "end_date":                     to_ts(res["time_end"]),
        "duration_days":                res["duration"],
        "intensity_max_degC":           res["intensity_max"],
        "intensity_mean_degC":          res["intensity_mean"],
        "cumulative_intensity_degC":    res["intensity_cumulative"],
    })

# Open files
ds = open_sst(FILES_GLOB, chunks={"time": 120}, engine="netcdf4")

# Subset ROI &  'sst'
ds_roi = subset_roi(ds, ROI, var=boxmean_var_name)
da = ds_roi[boxmean_var_name]

# Inspect metadata
print_metadata(ds_roi, var=boxmean_var_name)


In [ ]:
#check if marineHeatWaves package is working

t = pd.date_range("2000-01-01", periods=40, freq="D")
ords = np.array([d.toordinal() for d in t], dtype=int)
temp = 25 + np.sin(np.linspace(0, 6.28, 40)); temp[15:25] += 2

res, clim = mhw.detect(ords, temp, climatologyPeriod=[2000, 2000], pctile=90, minDuration=5, joinAcrossGaps=True) #hobdey's defination
print("events:", len(res.get("time_start", [])))

# Verifies that core libs are importable and compatible with marineHeatWaves.
from importlib import import_module
import numpy as np
import marineHeatWaves as mhw

want = ["numpy", "pandas", "xarray", "dask", "bottleneck","netCDF4", "cftime", "marineHeatWaves"]
vers = {}
problems = []

for m in want:
    try:
        import_module(m)
        try:
            vers[m] = version(m)
        except PackageNotFoundError:
            mod = import_module(m)
            vers[m] = getattr(mod, "__version__", "unknown")
    except Exception as e:
        problems.append((m, str(e)))
        vers[m] = "IMPORT FAILED"

print("Package versions detailes:")
for k in want:
    print(f"{k:16s} : {vers[k]}")
if problems:
    print("\n[ENV WARNING] Some packages failed to import:")
    for m, msg in problems:
        print(f" - {m}: {msg}")

# marineHeatWaves.detect accepts ordinal ints + 1D temp array
try:
    # make a synthetic 40-day series with a warm spell
    days = np.array([pd.Timestamp("2000-01-01") + pd.Timedelta(i, "D") for i in range(40)])
    ords = np.array([d.toordinal() for d in days], dtype=int)
    temp = 25 + np.sin(np.linspace(0, 6.28, 40))
    temp[15:25] += 2.0  # warm event
    res, clim = mhw.detect(ords, temp, climatologyPeriod=[1991, 2020], pctile=90, minDuration=5, joinAcrossGaps=True)
    print("\nmarineHeatWaves.detect is working correctly.")
    print(f"Detected events: {len(res.get('time_start', []))}")
except Exception as e:
    print("\n[ENV ERROR] marineHeatWaves.detect detection failed:", e)
